In [1]:
import pickle
import glob
import os
from tqdm import tqdm
import numpy as np
from collections import defaultdict

exp = 'distribution'

def read_exp_dir(ds, exp='table'):
    exp_dir = f'/BS/mlcysec2/work/hierarchical-certification/log/{ds}/{exp}'
    overall_dict = {}
    for file in tqdm(glob.glob(os.path.join(exp_dir, '*.pkl'))[:100], desc=f'reading from {exp_dir}'):
        d = pickle.load(open(file, 'rb'))
        filename = os.path.basename(file).replace('.pkl', '')
        new_d = {}
        new_d[filename] = d[filename]
        d = new_d
        for image_name, image_d in d.items():
            for model_type, model_d in image_d.items():
                if model_type not in overall_dict:
                    overall_dict[model_type] = {}
                for metric, value in model_d.items():
                    if isinstance(value, dict):
                        for k, v in value.items():
                            if isinstance(v, dict):
                                if metric not in overall_dict[model_type]:
                                    overall_dict[model_type][metric] = {}
                                if k not in overall_dict[model_type][metric]:
                                    overall_dict[model_type][metric][k] = {}
                                for k_, v_ in v.items():
                                    if k_ not in overall_dict[model_type][metric][k]:
                                        overall_dict[model_type][metric][k][k_] = v_
                                    else:
                                        overall_dict[model_type][metric][k][k_] += v_
                            else:
                                if k not in overall_dict[model_type]:
                                    overall_dict[model_type][k] = v
                                else:
                                    overall_dict[model_type][k] += v
                                
                        continue
                    if metric not in overall_dict[model_type]:
                        overall_dict[model_type][metric] = value
                    else:
                        overall_dict[model_type][metric] += value
    return overall_dict

dir_dict = {}
for ds in ['cityscapes', 'acdc', 'cocostuff', 'pascal_ctx']:
    dir_dict[ds] = read_exp_dir(ds, exp=exp)

reading from /BS/mlcysec2/work/hierarchical-certification/log/cityscapes/distribution: 100%|██████████| 100/100 [00:01<00:00, 98.86it/s]
reading from /BS/mlcysec2/work/hierarchical-certification/log/acdc/distribution: 100%|██████████| 100/100 [00:01<00:00, 82.70it/s]
reading from /BS/mlcysec2/work/hierarchical-certification/log/cocostuff/distribution: 100%|██████████| 100/100 [00:03<00:00, 31.12it/s]
reading from /BS/mlcysec2/work/hierarchical-certification/log/pascal_ctx/distribution: 100%|██████████| 100/100 [00:02<00:00, 34.31it/s]


In [5]:
d = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
n, n0, sigma, tau = 100, 10, 0.25, 0.75
num_classes = { 'cocostuff': 171, 'pascal_ctx': 59, 'cityscapes': 19, 'acdc': 19}
for ds, ds_d in dir_dict.items():
    for model_type, model_d in ds_d.items():
        if isinstance(model_type, tuple):
            n, n0, th, h_i, sigma, tau = model_type
            if (n, n0, sigma, tau) == (100, 10, 0.25, 0.75):
                if isinstance(th, str):
                    alg = 'AdaptiveCertify'
                if th is None:
                    alg = 'SegCertify'
                assert sum(model_d['num_pixels_per_cls_boundary']) == model_d['num_pixels_boundary']
                d[ds][alg]['%\\ abstain'] = model_d['abstain_count']/model_d['num_pixels']
                
                d[ds][alg]['%\\ abstain and boundary'] = model_d['abstain_count_boundary']/model_d['num_pixels_boundary']
                d[ds][alg]['%\\ abstain and non-boundary'] = (model_d['abstain_count'] - model_d['abstain_count_boundary'])/model_d['num_pixels_non_boundary']


                d[ds][alg]['%\\ boundary pixels'] = model_d['num_pixels_boundary']/model_d['num_pixels']
                d[ds][alg]['abstain %\\ of boundary pixels'] = model_d['abstain_count_boundary']/model_d['num_pixels_boundary']
 
                d[ds][alg]['CIG'] = sum(model_d['cig_per_cls'])/model_d['num_pixels']/np.log(num_classes[ds])
                d[ds][alg]['CIG of boundary pixels'] = sum(model_d['cig_per_cls_boundary'])/model_d['num_pixels_boundary']/np.log(num_classes[ds])
                d[ds][alg]['CIG of non-boundary pixels'] = (sum(model_d['cig_per_cls'])-sum(model_d['cig_per_cls_boundary']))/model_d['num_pixels_non_boundary']/np.log(num_classes[ds])


d = dict(d)

In [6]:
rows = ['%\\ abstain', '%\\ abstain and boundary', '%\\ abstain and non-boundary', 'CIG', 'CIG of boundary pixels', 'CIG of non-boundary pixels']

In [10]:
table = r"""
\begin{table}
\setlength\tabcolsep{4pt}
\caption{Table with Sequential Entries}
\centering
\begin{tabularx}{\textwidth}{@{} *{8}{>{\centering\arraybackslash}X} @{}}
\toprule
\multicolumn{2}{c}{ } & \multicolumn{3}{c}{\textbf{dataset1}} & \multicolumn{3}{c}{\textbf{dataset2}}\\
\midrule
 & & {\fontsize{8}{0}\textsc{SegCertify}} & {\fontsize{8}{0}\textsc{AdaptiveCertify}} &  $\Delta$ & {\fontsize{8}{0}\textsc{SegCertify}} & {\fontsize{8}{0}\textsc{AdaptiveCertify}} &  $\Delta$ \\
\midrule
$\oslash\%\downarrow$ & & v01 & v02  & v03 & v04 & v05 & v06 \\
$\oslash\%\text{boundary} \downarrow$  & & v11 & v12  & v13 & v14 & v15 & v16 \\
$\oslash\%\text{non-boundary} \downarrow$  & & v21 & v22  & v23 & v24 & v25 & v26 \\
$\cig\uparrow$  & & v31 & v32  & v33 & v34 & v35 & v36 \\
$\cig\text{-boundary}\uparrow$  & & v41 & v42  & v43 & v44 & v45 & v46 \\
$\cig\text{-non-boundary}\uparrow$ & & v51 & v52  & v53 & v54 & v55 & v56 \\
\bottomrule
\end{tabularx}
\end{table}
"""

In [11]:
col = 1 
ds_map = {'cityscapes': 'Cityscapes', 'acdc': 'ACDC', 'pascal_ctx': 'PASCAL-Context', 'cocostuff':'COCO-Stuff-10K'}
for i, ds in enumerate(['pascal_ctx', 'cocostuff']):
    ds_name = ds_map[ds]
    table = table.replace(f'dataset{i+1}', ds_name)
    for row, k in enumerate(rows):
        col = 1+ 3*i
        for alg in ['SegCertify', 'AdaptiveCertify', 'delta']:
            
            if alg == 'delta':
                if row < 3:
                    val = round(d[ds]['SegCertify'][k]*100, 1) - round(d[ds]['AdaptiveCertify'][k]*100, 1)
                else:
                    val = round(d[ds]['AdaptiveCertify'][k], 2) - round(d[ds]['SegCertify'][k], 2)

            else:
                val = d[ds][alg][k]
            if 'abstain' in k:
                if alg != 'delta':
                    val = val*100
                val = round(val, 1)
                val = "{:.1f}".format(val)
            else:
                val = round(val, 2)
                val = "{:.2f}".format(val)
            if alg == 'delta':
                val = r'\textcolor{ForestGreen}{$'+val +'$}'
            else:
                val = f'${val}$'
            table = table.replace(f'v{row}{col}', val)
            col +=1
                    
print(table)


\begin{table}
\setlength\tabcolsep{4pt}
\caption{Table with Sequential Entries}
\centering
\begin{tabularx}{\textwidth}{@{} *{8}{>{\centering\arraybackslash}X} @{}}
\toprule
\multicolumn{2}{c}{ } & \multicolumn{3}{c}{\textbf{PASCAL-Context}} & \multicolumn{3}{c}{\textbf{COCO-Stuff-10K}}\\
\midrule
 & & {\fontsize{8}{0}\textsc{SegCertify}} & {\fontsize{8}{0}\textsc{AdaptiveCertify}} &  $\Delta$ & {\fontsize{8}{0}\textsc{SegCertify}} & {\fontsize{8}{0}\textsc{AdaptiveCertify}} &  $\Delta$ \\
\midrule
$\oslash\%\downarrow$ & & $20.6$ & $16.1$  & \textcolor{ForestGreen}{$4.5$} & $21.7$ & $14.2$ & \textcolor{ForestGreen}{$7.5$} \\
$\oslash\%\text{boundary} \downarrow$  & & $26.8$ & $22.2$  & \textcolor{ForestGreen}{$4.6$} & $26.4$ & $19.2$ & \textcolor{ForestGreen}{$7.2$} \\
$\oslash\%\text{non-boundary} \downarrow$  & & $20.0$ & $15.5$  & \textcolor{ForestGreen}{$4.5$} & $20.5$ & $12.9$ & \textcolor{ForestGreen}{$7.6$} \\
$\cig\uparrow$  & & $0.55$ & $0.56$  & \textcolor{ForestGreen}{$0.0